This version now uses the new method of deriving the reference observations to be truely free of contaminating emission.

The notebook EtaCar2_read_OTF_3801_L0.6_references_decontam.ipynb tests the method in more detail and compares it for single OTF strips to the conventional method.

update 2/6/2019:
Inserted smoothing of Tsys: pixel-tp-pixel jumps of more than thresh (=50) are set to 1E7. The subsequent arpls smoother ignores these pixels and replaces the values with a smoothed value. In addition, the entire Tsys spectra are smoothed, reducing its noise contribution. The associated error is estimated to be below 1%.

update 3/19/2019:
updated to STO_v34 to retrieve the correct coordinates for the [NII]-beams. Previously was a missing cos(dec) term in applying the offsets.

update 4/22/2019
Ongoing tuning of the arPLS baseline retrieval parameters.
subversion might have the baseline correction method added to the end of the notebook name to differentiate between the versions.

update 5/21/2019
created a new cleaned Level 0.7 data product vm18. The cleaning now used the arPLS smoothing to determine which pixels need repair and replacement. Might work better than before since first results looked promising.

update 8/26/2019
The cleaning of the Level 0.7 data has been improved (?) again. The latest is a 2-step cleaning process, repSpike1D() added to STO2_v34, which was initiated to create a cleaned product in order for Youngmin's software to also run on the [NII] data. Further updates are still in the works.

update 8/28/2019
Added checking for mask entry before performing the baseline fit.

update 11/14/2019 and before
Added FITS file output of original format with additions.
Added anothe ALS fitting with different parameters since implemented version does not neccessarily remove the baseline???
In future, the ALS baseline fitting should be implemented with quality control to catch outliers, a.k.a., baseline fits that went wrong.

update 2/20/2020
Changed the ALS fitting of the resampled spectrum. Now the input spectrum has the masked pixels removed before the ALS fit in order to provide the correct fit and remove possible interference by spikes.

update 2/21/2020
Inserted another spike testing for the final spectrum after baseline fitting to flag bad pixels missed before.<br>
Updated and tuned the baseline correction options. <br>
Few additional minor changes.

In [1]:
%matplotlib nbagg
import os
import numpy as np
import glob
import sys
import ntpath
from ALSFitter import arpls, nanarpls
from STO2_v35 import *
from astropy.io import fits
from astropy import units as u
from astropy.coordinates import SkyCoord, FK5
from pylab import *
from scipy import signal
from scipy.stats import moment
from scipy import interpolate
import warnings
import matplotlib.pyplot as plt
import matplotlib.mlab as mlab
import pandas as pd
import inspect

print(inspect.stack()[0][1])
plt.rcParams['axes.formatter.useoffset'] = False
plt.rc("font", size=5)
warnings.filterwarnings("ignore")

from IPython.core.display import HTML
def css_styling():
    styles = open("./jupyter_custom.css", "r").read()
    return HTML(styles)
css_styling()


def repUpdate(rep, irep, scan, obsid, obstype, verbose=False):
    """
       updating the log (aka rep or report) of cleaned spike pixels
       rep:   existing global report
       irep:  new addendum to report (can be None!)
       scan:  scan number
       obsid: observation ID
       obstype: observation type: hot, load, uref, dref, sig
    """
    if irep!=None:
        irep['scan'] = scan
        irep['obsid'] = obsid
        irep['type'] = obstype

        if rep!=None: rep = np.vstack((rep,irep))
        else: rep = irep
            
    if verbose: print('repUpdate:  obstype: %s pixels repaired'%(obstype))

    # return the updated (or not) report
    return rep


# reading ordered sequence of data files
def getFiles(stobsid, enobsid, cdirnum, path):
    afiles = []
    stobsid = np.squeeze(stobsid.copy())
    enobsid = np.squeeze(enobsid.copy())
    for i in range(stobsid, enobsid+1, 1):
        aa = os.path.join(path,cdirnum,'OTF%s_%05i.fits'%(cdirnum,i))
        # check if exists and append
        if os.path.isfile(aa): afiles.append(aa)
    return afiles


def queryDir(path, cdirnum, search, gscan=0, lscan=99999):
    """
    Function to query a STO-2 level 0.7 data directory for files matching the "search" string
    Input:
        path:    path to data directory
        cdirnum: string of observation subdir number (with leading 0)
        search:  search string (e.g.: '*OTF*.fits')
        gscan:   integer minimum scan number (inclusive)
        lscan:   integer maximum scan number (inclusive)
    """
    #sname = os.path.join(sto2_path,cdirnum,'*OTF*.fits')
    sname = os.path.join(path, cdirnum, search)
    afiles = sorted(glob.glob(sname))
    n_files = len(afiles)
    asp = np.array([af.split('_')[1].split('.')[0] for af in afiles], dtype=np.float)
    sel = np.where((asp>=gscan)&(asp<=lscan))
    
    return np.array(afiles)[sel]



def resampleMask(mask, resp):
#     imask = np.squeeze(mask[:resp*(mask.size//resp)])
#     nmask = imask[::resp]
#     for i in range(1,resp,1):
#         nmask = np.bitwise_or(nmask, imask[i::resp])
    
#     return nmask
    if mask.ndim==1:
        imask = np.squeeze(mask[:resp*(mask.size//resp)])
        nmask = imask[::resp]
        for i in range(1,resp,1):
            nmask = np.bitwise_or(nmask, imask[i::resp])
        
        return nmask
    elif mask.ndim==2:
        imask = np.squeeze(mask[:resp*(mask.shape[0]//resp),:])
        nmask = imask[::resp]
        for i in range(1,resp,1):
            nmask = np.bitwise_or(nmask, imask[i::resp,:])
        
        return nmask
    else:
        return None



import time
s_today = time.strftime("%Y%m%d")
now = time.strftime("%c")
print ("Last execution: %s"  % now )

target = 'EtaCar'
ctarget = 'Eta Carinae'


# 
#
# v_LSR = -50 km/s
#
# zero position
# CDS position of EtaCar5:  ra=10 45 03.546 -59 41 03.95 
#pos0 = SkyCoord('10h45m03.546s', '−59d41m03.95s', frame='icrs')
# Eta Car: 286.10000,0.20000
# Eta Car: 10h37m56.1s -58d14m24.9s
#pos0 = SkyCoord(286.1*u.deg, 0.2*u.deg, frame='galactic').transform_to('fk5').transform_to(FK5(equinox='J2000'))
pos0 = SkyCoord('10h45m00s', '−59d41m', frame='icrs')   # approximately from Matsuo et al. 2009
l0 = pos0.galactic.l.degree
b0 = pos0.galactic.b.degree
ra0 = pos0.ra.deg
dec0 = pos0.dec.deg
print('Eta Car: %9.5f,%.5f'%(l0,b0))
print('Eta Car: %s'%(pos0.to_string('hmsdms', precision=1)))
    
dversion = 'vp36'
sto2_path = './Data/level0.7%s/'%(dversion)
ofileroot = './Data/processed/EtaCar_map1_0.7%s'%(dversion)


verbose = False
verbose2 = False

debug = False
plotdebug = False
# debug = False
# plotdebug = False
debug_stdir = 3859 # 3859

cleanflag = False
pcflag = False          # print the cleaning report
boff = 2
redflag = False
saveflag = True
if debug: saveflag = False
add = ''
tbadpix = np.zeros(0)
rms1 = None
rms1range = None
rms2 = None
rms2range = None
norep = False       # force no repair (even if m1 or m2 in excel spreadsheet)

Tcal = 300.

# the Level 0.7v data have a new index for the CII line
# select a line 0: NII_1, 1: NII_2, 2: CII_2 
lin = 2
trim = 0       # trimming the edges of the spectra
rclip = 3      # replacing outer pixels with next value
xlim  =[-50.,50.]
ylim = [-30, 30.]
anrange = [-99.0,49.] # range over which analysis should take place
#anrange = [-120.0,65.] # range over which analysis should take place
if (lin==0) | (lin==1): vrange = [-120., 65.] # limited velocity range for [NII]
else: vrange = None
rms1range = np.array([-70.,-30.])
rms2range = np.array([0.0, 40.])
resp = 4      # number of pixels to be resampled
    
# baseline fit exclusion region, e.g., presence of a line
bxrange = np.array([-35, -8])   # km/s
bxmode = 'slinear'   # from scipy.interpolate: ‘linear’, ‘nearest’, ‘zero’, ‘slinear’, ‘quadratic’, ‘cubic’, ‘previous’, ‘next’


# derive correction factor for sideband
# sideband gainratio R = g_usb / g_lsb => 
# fraction of from USB: G_usb = g_usb / (g_usb + g_lsb) = R / (1 + R) 
# for R = 1: G_USB = 0.5 OR Spectrum_USB = Spectrum_dsb / G_usb = 2.0 * Spectrum_dsb
calibtype = 'SSB' # desired calibration: single sideband SSB or double sideband DSB
gainratio = 1.0  # we do not know better
if calibtype=='SSB':
    sbcorr = (1 + gainratio) / gainratio
else:
    sbcorr = 1.0
calibunit = 'K'
caliblevel = 'T_A^*'


# directory range for map:
# the directories are alternating OTF and REF/HOT observations
# the obs span dirs 6647 to 6757
# the central position is around dirs 6680 and 6682
stdir = 3801    # 3549
endir = 3993  #3994    # 3563
if debug: 
    stdir = debug_stdir
    endir = stdir + 2

mapdirs = np.arange(stdir,endir,2)
#mapdirs = np.array([3981])         # for debugging only single strip
#if mapdirs.size==1: saveflag=False  # for debugging no saving of processed data   

# center of OTF strip is 19
obsids = np.arange(2,54,1)

if   lin==2: 
    cline = '[CII]'
    cline2 = '[CII]'
    add = '_CII_2'
    badpix = np.zeros(0)
elif lin==1: 
    cline = '[NII]'
    cline2 = '[NII]2'
    add = '_NII_2'
    badpix = np.zeros(0)    # bad pixels more or less in all scans
elif lin==0: 
    cline = '[NII]'
    cline2 = '[NII]1'
    add = '_NII_1'
    badpix = np.zeros(0)    # bad pixels more or less in all scans
else: add = ''
    
# if redflag:
#     add += '_red'
# else:
#     add += '_full'


print('Processing data for line: %s'%(add.replace('_',' ')))


ofile = ofileroot+add+'_%i_%s.fits'%(stdir, s_today)
nfile = ofileroot+add+'_%i_%s.npz'%(stdir, s_today)   # numpy archive file name


# spectrum
n_pix = 1024    # number of pixels per spectrum



################################################################################
# get the predetermined info about the OTF scans
print('reading excel data analysis sheet.')

xfile = './Data/Ref_Lists/etaCar_reference_scans_list4.xlsx'
with pd.ExcelFile(xfile) as xlsx:
    sot = pd.read_excel(xlsx, 1, header=0, engine = 'xlrd')

sotsz = sot.shape
sot = sot.tail(sotsz[0]-1)
colnames = sot.columns
print(colnames)
['drudir', '    dru', 'dhudir', '   dhu', 'dhucor', 'ohudir', '   ohu', 'orudir', '   oru', 'orucor', 'spdir', '  spec', ' spec2', 'orddir',
   '   ord', 'ordcor', 'ohddir', '   ohd', 'dhddir', '   dhd', 'drddir', '   drd', 'des ref1', 'des ref2', 'brange1[0]', 'brange1[1]',
   'brange2[0]', 'brange2[1]', 'order1', 'order2', 'despike']
# handle is "1" for the scans to be processed
so_spec  = np.array(sot['  spec'], dtype=np.int)
so_spec2 = np.array(sot[' spec2'], dtype=np.int)
so_spsc  = np.array(sot['spdir'], dtype=np.int)
nrows = so_spsc.size
print(so_spec.shape)

# otf reference upper
so_oru = np.array(sot['   oru'], dtype=np.int)
so_orud = np.array(sot['orudir'], dtype=np.int)
so_ohu = np.array(sot['   ohu'], dtype=np.int)
so_ohud = np.array(sot['ohudir'], dtype=np.int)
# designated reference upper
so_dru = np.array(sot['    dru'], dtype=np.int)
so_drud = np.array(sot['drudir'], dtype=np.int)
so_dhu = np.array(sot['   dhu'], dtype=np.int)
so_dhud = np.array(sot['dhudir'], dtype=np.int)
# otf reference down
so_ord = np.array(sot['   ord'], dtype=np.int)
so_ordd = np.array(sot['orddir'], dtype=np.int)
so_ohd = np.array(sot['   ohd'], dtype=np.int)
so_ohdd = np.array(sot['ohddir'], dtype=np.int)
# designated reference down
so_drd = np.array(sot['   drd'], dtype=np.int)
so_drdd = np.array(sot['drddir'], dtype=np.int)
so_dhd = np.array(sot['   dhd'], dtype=np.int)
so_dhdd = np.array(sot['dhddir'], dtype=np.int)

so_case1 = np.array(sot['des ref1'], dtype=np.object)
so_case2 = np.array(sot['des ref2'], dtype=np.object)

so_br1 = np.zeros([nrows,2])
so_br1[:,0] = np.array(sot['brange1[0]'], dtype=np.float)
so_br1[:,1] = np.array(sot['brange1[1]'], dtype=np.float)
so_bo1 = np.array(sot['order1'], dtype=np.float)
so_br2 = np.zeros([nrows,2])
so_br2[:,0] = np.array(sot['brange2[0]'], dtype=np.float)
so_br2[:,1] = np.array(sot['brange2[1]'], dtype=np.float)
so_bo2 = np.array(sot['order2'], dtype=np.float)

so_order1 = np.array(sot['order1'], dtype=np.object)
so_order2 = np.array(sot['order2'], dtype=np.object)

so_despike = np.array(sot['despike'], dtype=np.object)


badotfs = np.array([3905,3907])
mapdirs = mapdirs[np.isin(mapdirs, badotfs, invert=True),]

cnt = 0
#if mapdirs.size==1: cnt = np.argwhere(so_spsc==mapdirs[0])[0][0]
init = True
print('mapdirs: ', mapdirs)
if len(mapdirs)==1:
    # set the counter so we can process only a single spectrum
    cnt = np.squeeze(np.argwhere(so_spsc==mapdirs[0]))
    debug = True

for j in range(len(mapdirs)):
        
    #Signal
    dirnum = mapdirs[j]
    cdirnum = '%05i'%(dirnum)
    chdirnum1 = '%05i'%(dirnum-1)
    chdirnum2 = '%05i'%(dirnum+1)

    repflag = np.zeros((3), dtype=int)
    repmode = np.array(['org','org','org'])   # repmode: org: original, pre (previous HOT used), post (later HOT used), avg (average of pre and post HOT)
    
    if so_spsc[cnt]!=dirnum:   # e.g., scan 3905 is bad
        print('Warning: problem in scan sequence. Expecting scan %i, but loop at scan %i.'%(so_spsc[cnt], dirnum))
        cnt += 1
        #continue
        
#     if dirnum in badotfs:
#         print('Warning: Bad OTF scan. Skipping scan %i.'%(dirnum))
#         continue
    
    print()
    print('processing OTF scan : %s with reference scans dirs: %s and %s'%(cdirnum, chdirnum1, chdirnum2))
    #sname = os.path.join(sto2_path,cdirnum,'*OTF*.fits')
    #afiles = sorted(glob.glob(sname))
    # we want only a reduced set of the OTF scans as pre-determined!
    if debug: print('scan ids: ', cdirnum, so_spec[cnt], so_spec2[cnt])
    #afiles = getFiles(so_spec[cnt], so_spec2[cnt], cdirnum, sto2_path)
    afiles = getFiles(so_oru[cnt], so_ord[cnt], cdirnum, sto2_path)   # try to get more spectra!
    n_files = len(afiles)
    #for i in range(n_files): print(afiles[i])
    
    ###############################################################################################
    # determine Tsys
    # we get a Tsys from either end of the OTF strip.
    # If a TSYS does not exist, we try to recover it by averaging the before 
    # and next Tsys from the same OTF strip end. There might also be some mitigation 
    # coming from the excel spreadsheet list read above.
    
    cTsys1 = 'Tsys 1'
    t1rec = 0
    Tsys1, tvv1, info1, rep1, t1mask = getTsysLine(chdirnum1, lin, ipath=sto2_path, verbose=verbose2, badpix=tbadpix, rclip=rclip, return_mask=True)
    t1bad = np.all(t1mask>0)
    if ((np.nanmean(Tsys1.value)==np.nanmax(Tsys1.value))|(np.nanmean(Tsys1.value)>3000.)|(np.nanmean(Tsys1.value)<1000.)):
        if verbose: print(np.nanmax(Tsys1.value),np.nanmin(Tsys1.value), Tsys1.value[50:350])
        # Tsys2 is not correct, try to recover
        chdirnum1m = '%05i'%(dirnum-1-4)
        Tsys11, tvv11, info11, rep11, t11mask = getTsysLine(chdirnum1m, lin, ipath=sto2_path, verbose=verbose2, badpix=tbadpix, rclip=rclip, return_mask=True)
        chdirnum1m = '%05i'%(dirnum-1+4)
        Tsys12, tvv12, info12, rep12, t12mask = getTsysLine(chdirnum1m, lin, ipath=sto2_path, verbose=verbose2, badpix=tbadpix, rclip=rclip, return_mask=True)
        if np.nanmean(Tsys11.value)>100: fc11 = 1.
        else: fc11 = 0.
        if np.nanmean(Tsys12.value)>100: fc12 = 1.
        else: fc12 = 0.
        Tsys1 = (fc11*Tsys11 + fc12*Tsys12) / (fc11+fc12)
        tvv1 = (fc11*tvv11 + fc12*tvv12) / (fc11+fc12)
        t1mask = np.bitwise_or(t11mask, t12mask)
        cTsys1 = 'Tsys 1 recov.'
        t1rec = 1
    Tsky = info1['Tsky']
    Thot = info1['Thot']
    
    cTsys2 = 'Tsys 2'
    t2rec = 0
    Tsys2, tvv2, info2, rep2, t2mask = getTsysLine(chdirnum2, lin, ipath=sto2_path, verbose=verbose2, badpix=tbadpix, rclip=rclip, return_mask=True)
    t2bad = np.all(t2mask>0)
    if ((np.nanmean(Tsys2.value)==np.nanmax(Tsys2.value))|(np.nanmean(Tsys2.value)>3000.)|(np.nanmean(Tsys2.value)<1000.)):
        # Tsys2 is not correct, try to recover
        chdirnum2m = '%05i'%(dirnum+1-4)
        Tsys21, tvv21, info21, rep21, t21mask = getTsysLine(chdirnum2m, lin, ipath=sto2_path, verbose=verbose2, badpix=tbadpix, rclip=rclip, return_mask=True)
        chdirnum2m = '%05i'%(dirnum+1+4)
        Tsys22, tvv22, info22, rep22, t22mask = getTsysLine(chdirnum2m, lin, ipath=sto2_path, verbose=verbose2, badpix=tbadpix, rclip=rclip, return_mask=True)
        if np.nanmean(Tsys21.value)>100: fc21 = 1.
        else: fc21 = 0.
        if np.nanmean(Tsys22.value)>100: fc22 = 1.
        else: fc22 = 0.
        Tsys2 = (fc21*Tsys21 + fc22*Tsys22) / (fc21+fc22)
        tvv2 = (fc21*tvv21 + fc22*tvv22) / (fc21+fc22)
        t2mask = np.bitwise_or(t21mask, t22mask)
        cTsys2 = 'Tsys 2 recov.'
        t2rec = 1
                
    if debug: print('mean Tsys1: ', np.nanmean(Tsys1))
    if debug: print('mean Tsys2: ', np.nanmean(Tsys2))
    rep1 = 'o'
    rep2 = 'o'
    if np.isnan(np.nanmean(Tsys1)):
        # print('Tsys1 is nan. replaced')
        Tsys1 = Tsys2
        rep1 = 'r'
    if np.isnan(np.nanmean(Tsys2)):
        #print('Tsys2 is nan. replaced')
        Tsys2 = Tsys1
        rep2 = 'r'
    
    # smooth the Tsys using arpls
    Tsys1o = Tsys1.copy()
    Tsys2o = Tsys2.copy()
    Tsys1o.value[np.where(np.isnan(Tsys1o.value))] = 1E7
    Tsys2o.value[np.where(np.isnan(Tsys2o.value))] = 1E7
    Tsys1o.value[np.where(np.isinf(Tsys1o.value))] = 1E7
    Tsys2o.value[np.where(np.isinf(Tsys2o.value))] = 1E7
    Tsys1o.value[Tsys1o.value<1000.] = 1E7
    Tsys2o.value[Tsys2o.value<1000.] = 1E7
    # remove smaller spikes and smooth them
    thresh = 50.
    Tsys1o.value[np.where(np.abs((Tsys1o.value[1:]-Tsys1o.value[:-1]))>thresh)] = 1E7
    Tsys2o.value[np.where(np.abs((Tsys2o.value[1:]-Tsys2o.value[:-1]))>thresh)] = 1E7
    # the ratio has been tuned and lowered from 0.01 to 0.001 since it was causing spikes.
    Tsys1, w1 = arplsw(Tsys1o.value, lam=2, ratio=0.001, itermax=30) * u.K
    Tsys2, w2 = arplsw(Tsys2o.value, lam=2, ratio=0.001, itermax=30) * u.K

    tostr = 'mean Tsys1 and Tsys2: %.1f K (%s)  %.1f K (%s)'%(np.nanmean(Tsys1.value), rep1, np.nanmean(Tsys2.value), rep2)
    if debug: 
        print(tostr)
        fig = plt.figure()
        sp1 = plt.subplot()
        pl1 = plt.plot(Tsys2o, color='lightgreen')
        pl2 = plt.plot(Tsys1o, color='skyblue')
        pl1 = plt.plot(Tsys2, color='green')
        pl2 = plt.plot(Tsys1, color='blue')

 
    ##############################################################################################    
    # start processing the data
    
    ###### reference data (taken before OTF strip, also used for Tsys calculation):
    # designated reference upper
    drufile = os.path.join(sto2_path,'%05i/REF%05i_%05i.fits'%(so_drud[cnt],so_drud[cnt],so_dru[cnt]))
    drul = 'rref'
    if verbose2: print('designated ref: ', drufile)
    druvv, dru, drumask, drupos, druh1 = readSTO2LineM(drufile, lin, verbose=verbose, rclip=rclip, badpix=badpix)
    drubad = np.all(drumask>0)
    druTint  = np.float(druh1['OBSTIME'])
    druobsid = np.float(druh1['OBSID'])
    drutime  = np.float(druh1['UNIXTIME'])
    
    # in case we have to repair the REF spectrum 
    if (so_case1[cnt]=='m1')&(norep==False):
        # the designated ref 1 is missing !!!! 
        # => try to get the missing reference from old and future reference observations and average, in case they exist
        chdirnum1m = '%05i'%(dirnum-1-4)
        hun1 = os.path.join(sto2_path,chdirnum1m,'*REF*.fits')
        druf1 = sorted(glob.glob(hun1))[0]
        druvv1, dru1, dru1mask, drupos1, druh11 = readSTO2LineM(druf1, lin, verbose=verbose, rclip=rclip, badpix=badpix)
        if np.nanmean(dru1.value)>1E4: fc1 = 1.
        else: fc1 =0.
        chdirnum1m = '%05i'%(dirnum-1+4)
        hun2 = os.path.join(sto2_path,chdirnum1m,'*REF*.fits')
        druf2 = sorted(glob.glob(hun2))[0]
        druvv2, dru2, dru2mask, drupos2, druh12 = readSTO2LineM(druf2, lin, verbose=verbose, rclip=rclip, badpix=badpix)
        if np.nanmean(dru2.value)>1E4: fc2 = 1.
        else: fc2 =0.
        if (fc1==0.)&(fc2==0.): print('Error. No reference observation available for scan %s'%(cdirnum))
        dru = (fc1 * dru1 + fc2 * dru2) / (fc1 + fc2)
        druvv = (fc1 * druvv1 + fc2 * druvv2) / (fc1 + fc2)
        if (fc1==0.): 
            repmode = 'post'
            drumask = dru2mask
        if (fc2==0.): 
            repmode = 'pre'
            drumask = dru1mask
        else: 
            repmode = 'avg'
            drumask = np.bitwise_or(dru1mask, dru2mask)
        repflag[lin] = 1      # set flag since data were repaired
        drul = 'rrefm'

    
    # designated hot upper
    dhufile = os.path.join(sto2_path,'%05i/HOT%05i_%05i.fits'%(so_dhud[cnt],so_dhud[cnt],so_dhu[cnt]))
    dhul = 'rhot'
    dhuvv, dhu, dhumask, dhupos, dhuh1 = readSTO2LineM(dhufile, lin, verbose=verbose, rclip=rclip, badpix=badpix)
    dhubad = np.all(dhumask>0)
    dhuTint  = np.float(dhuh1['OBSTIME'])
    dhuobsid = np.float(dhuh1['OBSID'])
    dhutime  = np.float(dhuh1['UNIXTIME'])



    # data in OTF strip scan
    # OTF hot upper
    ohufile = os.path.join(sto2_path,'%05i/HOT%05i_%05i.fits'%(so_ohud[cnt],so_ohud[cnt],so_ohu[cnt]))
    ohul = 'ohot'
    ohuvv, ohu, ohumask, ohupos, ohuh1 = readSTO2LineM(ohufile, lin, verbose=verbose, rclip=rclip, badpix=badpix)
    ohubad = np.all(ohumask>0)
    ohuTint  = np.float(ohuh1['OBSTIME'])
    ohuobsid = np.float(ohuh1['OBSID'])
    ohutime  = np.float(ohuh1['UNIXTIME'])
    if ohu.mean().value <=1.: 
        ohu = dhu
        if verbose: print('ALERT!!!!: otf hot 0, ohu, was replaced with ref hot, dhu')

    
    # otf reference upper (start of OTF scan)
    orufile = os.path.join(sto2_path,'%05i/OTF%05i_%05i.fits'%(so_orud[cnt],so_orud[cnt],so_oru[cnt]))
    orul = 'af%i'%(so_oru[cnt])
    if verbose2: print('upper ref: ', orufile)
    oruvv, oru, orumask, orupos, oruh1 = readSTO2LineM(orufile, lin, verbose=verbose, rclip=rclip, badpix=badpix)
    orubad = np.all(orumask>0)
    oruTint  = np.float(oruh1['OBSTIME'])
    oruobsid = np.float(oruh1['OBSID'])
    orutime  = np.float(oruh1['UNIXTIME'])


    ##### OTF Signal Spectra 
    # get the OTF data files according to the spreadsheet, which lists the first and the last scan to be used
    # hd1['SPECSTA4']=specsta4=='0x2000001' => spectrum is useful
    if debug: print('scan ids: ', cdirnum, so_spec[cnt], so_spec2[cnt])
    afiles = getFiles(so_spec[cnt], so_spec2[cnt], cdirnum, sto2_path)


    # OTF reference down - ord (end of OTF strip)
    ordfile = os.path.join(sto2_path,'%05i/OTF%05i_%05i.fits'%(so_ordd[cnt],so_ordd[cnt],so_ord[cnt]))
    ordl = 'af%i'%(so_ord[cnt])
    ordvv, ord, ordmask, ordpos, ordh1 = readSTO2LineM(ordfile, lin, verbose=verbose, rclip=rclip, badpix=badpix)
    ordbad = np.all(ordmask>0)
    ordTint  = np.float(ordh1['OBSTIME'])
    ordobsid = np.float(ordh1['OBSID'])
    ordtime  = np.float(ordh1['UNIXTIME'])

    # OTF hot down - ohd
    ohdfile = os.path.join(sto2_path,'%05i/HOT%05i_%05i.fits'%(so_ohdd[cnt],so_ohdd[cnt],so_ohd[cnt]))
    ohdl = 'ohot2'
    ohdvv, ohd, ohdmask, ohdpos, ohdh1 = readSTO2LineM(ohdfile, lin, verbose=verbose, rclip=rclip, badpix=badpix)
    ohdbad = np.all(ohdmask>0)
    ohdTint  = np.float(ohdh1['OBSTIME'])
    ohdobsid = np.float(ohdh1['OBSID'])
    ohdtime  = np.float(ohdh1['UNIXTIME'])

    
    
    ##### reference data (taken after OTF strip, also used for Tsys calculation):
    # designated reference down - drd
    drdfile = os.path.join(sto2_path,'%05i/REF%05i_%05i.fits'%(so_drdd[cnt],so_drdd[cnt],so_drd[cnt]))
    drdl = 'rref2'
    drdvv, drd, drdmask, hdpos, drdh1 = readSTO2LineM(drdfile, lin, verbose=verbose, rclip=rclip, badpix=badpix)
    drdbad = np.all(drdmask>0)
    drdTint  = np.float(drdh1['OBSTIME'])
    drdobsid = np.float(drdh1['OBSID'])
    drdtime  = np.float(drdh1['UNIXTIME'])

    if (so_case2[cnt]=='m2')&(norep==False):
        if verbose: print('Case: m2')
        # the designated ref 1 is missing !!!! 
        # => try to get the missing reference from old and future reference observations and average, in case they exist
        chdirnum2m = '%05i'%(dirnum+1-4)
        hdn1 = os.path.join(sto2_path,chdirnum2m,'*REF*.fits')
        drdf1 = sorted(glob.glob(hdn1))[0]
        drdvv1, drd1, drd1mask, drdpos1, drdh11 = readSTO2LineM(drdf1, lin, verbose=verbose, rclip=rclip, badpix=badpix)
        if np.nanmean(drd1.value)>1E4: fc12 = 1.
        else: fc12 =0.
        chdirnum2m = '%05i'%(dirnum+1+4)
        hdn2 = os.path.join(sto2_path,chdirnum2m,'*REF*.fits')
        drdf2 = sorted(glob.glob(hdn2))[0]
        drdvv2, drd2, drd2mask, drdpos2, drdh12 = readSTO2LineM(drdf2, lin, verbose=verbose, rclip=rclip, badpix=badpix)
        if np.nanmean(drd2.value)>1E4: fc22 = 1.
        else: fc22 =0.
        if (fc12==0.)&(fc22==0.): print('Error. No reference observation available for scan %s'%(cdirnum))
        drd = (fc12 * drd1 + fc22 * drd2) / (fc12 + fc22)
        drdvv = (fc12 * drdvv1 + fc22 * drdvv2) / (fc12 + fc22)
        if (fc12==0.): 
            repmode = 'post'
            drdmask = drd2mask
        if (fc22==0.): 
            repmode = 'pre'
            drdmask = drd1mask
        else: 
            repmode = 'avg'
            drdmask = np.bitwise_or(drd1mask, drd2mask)
        repflag[lin] = 1      # set flag since data were repaired
        drdl = 'rref2m'


    # designated hot down - dhd spectrum
    dhdfile = os.path.join(sto2_path,'%05i/HOT%05i_%05i.fits'%(so_dhdd[cnt],so_dhdd[cnt],so_dhd[cnt]))
    dhdl = 'rhot2'
    dhdvv, dhd, dhdmask, dhdpos, dhdh1 = readSTO2LineM(dhdfile, lin, verbose=verbose, rclip=rclip, badpix=badpix)
    dhdbad = np.all(dhdmask>0)
    dhdTint  = np.float(dhdh1['OBSTIME'])
    dhdobsid = np.float(dhdh1['OBSID'])
    dhdtime  = np.float(dhdh1['UNIXTIME'])

    # done reading data
    #########################################################################################################
    

 
    ##############################################################################################    
    # Start processing the data
    # (1) We are using the designated references and interpolate between start and end of OTF scan to 
    # mitigate linear drifts.
    # (2) We are also using the HOT-measurement to further mitigate drifts.
    #
    # SIGi = <Tsys>t * (OTFi/<HOT>t - <REF>t/<HOT>t / (<REF>t/<HOT>t))
    #
    # <x>t: means time-weighted average of quantity x
    # i:    i-th astronomical observation of OTF scan
    # 
    # Now, how to correct for the remaining drift in the scans.
    # (a) posible method is to use the data from the spreadsheet that includes a reference scan at the 
    # start and at the end of the OTF strip and the sequence of signal measurements. The references then 
    # can be baseline fitted and the baseline is corrected for in the raw spectra. These two corrected 
    # reference spectra can then be used to calculate <REF>t and used with the above equations to derive 
    # the final signal spectrum. Remaining baseline wiggles can then be removed with another (low-order)
    # polynomial baseline fit.
    
    # define data record for each final spectrum that includes all the relevant information
    # these records are then stacked and saved
    oscan0 = {}
    
    # We have to create a correction derived from the baseline fit to the first and last spectrum in each OTF scan.
    # Then, this baseline correction can be treated like the reference scans: 
    #     use the proper fractions from either end of the OTF scan and and removed the result from the scans
    #
    svv1, spec1, smask1, spos1, sh11 = readSTO2LineM(afiles[0],  lin, verbose=verbose, rclip=rclip, badpix=badpix)
    if debug: print('smask1: ', smask1.shape, lin, smask1[0:500])
    rtime1  = np.float(sh11['UNIXTIME'])
    svv2, spec2, smask2, spos2, sh12 = readSTO2LineM(afiles[-1], lin, verbose=verbose, rclip=rclip, badpix=badpix)
    rtime2  = np.float(sh12['UNIXTIME'])
    if debug: print('afiles[0]:  ', afiles[0])
    if debug: print('afiles[-1]: ', afiles[-1])
    
    # 1. get the initial calibrated spectra
    csp1 = (spec1/ohu - dru/dhu) / (dru/dhu) * Tsys1
    csp2 = (spec2/ohd - drd/dhd) / (drd/dhd) * Tsys2
    if debug: print('test: ', spec2[100:130], drd[100:130])
    
    # 2. resample the spectra
    svv1h, csp1h = resampleSpectrum(svv1, csp1, resp)
    svv2h, csp2h = resampleSpectrum(svv2, csp2, resp)
    if debug: print('smask1: ', smask1.shape, smask1[10:30])
    smask1h = resampleMask(smask1, resp)
    smask2h = resampleMask(smask2, resp)
    if debug: 
        print('smask1 resampled: ', smask1.shape, smask1[10:30])
        print('smask2 resampled: ', smask2.shape, smask2[10:30])
    
    # 3. reduce the spectra to the analysis range
    asel = np.where((svv1h.value>=anrange[0])&(svv1h.value<=anrange[1]))
    svv1ha = svv1h[asel]
    csp1ha = csp1h[asel]
    svv2ha = svv2h[asel]
    csp2ha = csp2h[asel]
    smask1ha = smask1h[asel]
    smask2ha = smask2h[asel]

    
    # 4. baseline fit to the spectra from # 
    # 4.0 Prepare data and polynomial fits
    brange1 = so_br1[cnt,:]  
    #print('brange1: ', brange1)
    if np.any(np.isnan(brange1)): brange1 = [0.,0.]
    if np.isfinite(so_order1[cnt]): order1 = so_order1[cnt]  
    else: order1 = 19
        
    brange2 = so_br2[cnt,:]  
    if np.any(np.isnan(brange2)): brange2 = [0.,0.]
    if np.isfinite(so_order2[cnt]): order2 = so_order2[cnt]  
    else: order2 = 19
    
    #### 4.1 baseline 1
    #print('mask: ', smask1ha)
    bsel1 = np.where(((svv1ha.value<brange1[0])|(svv1ha.value>brange1[1]))&(smask1ha==0)&np.isfinite(csp1ha))[0]
    bsp1ha = csp1ha[bsel1].value
    bvv1ha = svv1ha[bsel1].value
    zu1 = np.polyfit(bvv1ha, bsp1ha, order1)
    pu1 = np.poly1d(zu1)
    bsp1 = csp1.value - pu1(svv1.value)              # baseline corrected full spectrum
    bcor1 = pu1(svv1.value)                          # baseline correction only, for full spectrum
    bcor1h = pu1(svv1h.value)                        # baseline correction only, for resampled spectrum
    bcor1ha = pu1(svv1ha.value)                      # baseline correction only, for resampled reduced spectrum
    bsp1s = smoothData(bsp1, fs=2000)*u.K      # basically this is the smooth contamination
    
    # now, we have to reverse de-calibrate the error signal in csp1habs
    bsp1str = bsp1s / Tsys1 * (dru/dhu) * ohu
    # control calculation
    csp1c = ((spec1-bsp1str)/ohu - dru/dhu) / (dru/dhu) * Tsys1    # start of OTF scan
    z1 = np.zeros(csp1.size)*u.K
    ref1 = (((z1+bcor1*u.K)*(dru/dhu)/Tsys1)+(dru/dhu))*ohu
    nsp1 = (spec1 - ref1) / ref1 * Tsys1      # this still might require the polyfit.
    
    
    
    #### 4.2 baseline 2
    bsel2 = np.where(((svv2ha.value<brange2[0])|(svv2ha.value>brange2[1]))&(smask2ha==0)&np.isfinite(csp2ha))[0]
    bsp2ha = csp2ha[bsel2].value
    bvv2ha = svv2ha[bsel2].value
    zu2 = np.polyfit(bvv2ha, bsp2ha, order2)
    pu2 = np.poly1d(zu2)
    bcor2 = pu2(svv2.value)                          # baseline correction only, for full spectrum
    bsp2 = csp2.value - pu2(svv2.value)              # baseline corrected full spectrum
    bsp2s = smoothData(bsp2, fs=2000)*u.K      # basically this is the smooth contamination
    
    # now, we have to reverse de-calibrate the error signal in csp1habs
    # this is the correction applied (subtracted) from the raw signal observation if used as reference
    bsp2str = bsp2s / Tsys2 * (drd/dhd) * ohd
    # control calculation
    csp2c = ((spec2-bsp2str)/ohd - drd/dhd) / (drd/dhd) * Tsys2
    z2 = np.zeros(csp2.size)*u.K
    ref2 = (((z2+bcor2*u.K)*(drd/dhd)/Tsys2)+(drd/dhd))*ohd
    nsp2 = (spec2 - ref2) / ref2 * Tsys2      # this still might require the polyfit.
    
    if plotdebug:
        print('Plots of csp1, bsp1, and nsp1')
        fig = plt.figure(figsize=(10,5))
        plt.plot(svv1, csp1, drawstyle='steps-post', color='blue', label='csp1')
        plt.plot(svv1, bsp1, drawstyle='steps-post', color='green', label='bsp1')
        plt.plot(svv1, nsp1, drawstyle='steps-post', color='red', label='nsp1')
        plt.ylim(-8,15)
        plt.legend()
        print(spec1[100], ref1[100], nsp1[100])
        
        print('Plots of csp2, bsp2, and nsp2')
        fig = plt.figure(figsize=(10,5))
        plt.plot(svv2, csp2, drawstyle='steps-post', color='blue', label='csp2')
        plt.plot(svv2, bsp2, drawstyle='steps-post', color='green', label='bsp2')
        plt.plot(svv2.value+0.01, nsp2, drawstyle='steps-post', color='red', label='nsp2')
        plt.ylim(-8,15)
        plt.legend()
        print(spec2[100], ref2[100], nsp2[100])
        
        print('Plots of Tsys')
        fig = plt.figure(figsize=(10,5))
        #plt.plot(svv, Tsys1, drawstyle='steps-post', color='blue', label='Tsys1')
        plt.plot(Tsys2, drawstyle='steps-post', color='green', label='Tsys2')
        plt.legend()
        
        print('Plots of bcor2')
        fig = plt.figure(figsize=(10,5))
        #plt.plot(svv, Tsys1, drawstyle='steps-post', color='blue', label='Tsys1')
        plt.plot(svv2ha, csp2ha, drawstyle='steps-post', color='blue', label='csp2ha')
        #plt.plot(bvv2ha, bsp2ha, drawstyle='steps-post', color='green', label='bsp2ha')
        plt.plot(svv2, bcor2, drawstyle='steps-post', color='red', label='bcor2')
        plt.legend()
        
        print('Plots of end of OTF')
        fig = plt.figure(figsize=(10,5))
        plt.plot(drd, drawstyle='steps-post', color='blue', label='drd')
        plt.plot(dhd, drawstyle='steps-post', color='red', label='dhd')
        plt.plot(ohd, drawstyle='steps-post', color='green', label='ohd')
        plt.plot(z2, drawstyle='steps-post', color='orange', label='z2')
        plt.plot(bcor2, drawstyle='steps-post', color='purple', label='bcor2')
        #plt.plot(svv2,Tsys2, drawstyle='steps-post', color='green', label='Tsys2')
        plt.ylim(-8,15)
        plt.legend()
        
        print('Now, the OTF spectra:')

    
    for afile in afiles:
        svv, spec, smask, spos, sh1, srs = readSTO2LineM(afile, lin, verbose=verbose, rclip=rclip, badpix=badpix, retcl=True)
        sbad = np.all(smask>0)
        sobsid = np.float(sh1['OBSID'])
        sTint  = np.float(sh1['OBSTIME'])
        stime  = np.float(sh1['UNIXTIME'])
        scan   = np.float(sh1['SCAN'])
        specsta4 = sh1['SPECSTA4']
        gl = spos.galactic.l.degree
        gb = spos.galactic.b.degree
        cmask = np.zeros([1024,16], dtype=np.int)  # mask for pixels
        amask = np.zeros([16], dtype=np.int)       # mask for spectrum,

        if ((sTint>0.)&(druTint>0.)&(drdTint>0.)&(dhuTint>0.)&(dhdTint>0.)&(ohuTint>0.)&(ohdTint>0.)&(specsta4=='0x2000001')):
            # determine the reference spectrum to be applied to spectrum
            # start: fraction is small (>= 0); end: fraction is large (<= 1)
            if drutime<drdtime:
                frac = (stime-drutime)/(drdtime-drutime)    # this is the fraction between the designated references
            else: # some mitigation if reusing references
                frac = (stime-dhutime)/(dhdtime-dhutime)
            frac2 = (stime-rtime1)/(rtime2-rtime1)          # this is the fraction between the references
            #print(rdobsid, obsid, ruobsid, frac)
            # the calibration as of now is DSB !
            Tsys = (1-frac2) * Tsys1 + frac2 * Tsys2
            sref = (1-frac2) * ref1  + frac2 * ref2     
            ################# signal spectrum #######################
            sig = (sbcorr * (spec - sref) / sref * Tsys).value
            
#             # for simplicity here, we use only the spectrum mask since there are no rules yet 
#             # how to combine the other masks for the final spectrum
#             print(smask.shape, smask.min(), smask.max())
#             cmask[:,0] = smask
            

            
            # added 3/4/2019
            # perform a last baselinefit: 
            # 1: resample spectrum to reduce noise
            # 2: check for nan's and infinite pixel values and 
            #    set them to the previous value or the next value depending on which one can be used (is finite)
            # 3: perform the baseline fit
            # 4: expand the fitted baseline to the full resolution by interpolating/extrapolating
            # 5: save the results 
            vbrange  = np.array([-120, 65.]) # velocity range for baseline fit; ful range: ~-120 to ~65 km/s
            vbrange  = np.array([-70, 45.])
            vbrange  = np.array([-95, 45.])
            rsel = np.where((svv.value>=vbrange[0])&(svv.value<=vbrange[1]))
                        
            # added 2/21/2020
            ulimit = 30.
            llimit = -15.
            off = np.mean(np.percentile(sig, 60))
            smaskr = np.squeeze(smask[rsel])
            osel = np.where( (((sig[rsel]-off) > ulimit)|((sig[rsel]-off) < llimit)) & smaskr<1)
            if debug: print(off, (sig[rsel]-off).min(), (sig[rsel]-off).max(), (sig[rsel]-off).mean(), np.median((sig[rsel]-off)), smask[osel].size)
            (smask[rsel])[osel] = 32
            if debug: print('smask: ', smask.shape, smask[osel].size)
            
            
            svvr, sspr = resampleSpectrum(svv[rsel].value, sig[rsel], 4)
            ssprb = np.squeeze(sspr.copy())
            smsk = resampleMask(smask[rsel], 4)
            sspr[smsk>0] = np.nan
            ssprb[smsk>0] = np.nan
            if debug: print(sspr.shape, smsk.shape, smask.shape)
            #nsel = np.where(np.isnan(sspr)|np.isinf(sspr)|np.sum(smsk,axis=1)>0)
            nsel = np.where(np.isnan(sspr)|np.isinf(sspr)|smsk>0)
            if len(nsel)>0:
                for km in nsel[0]:
                    try:
                        if np.isfinite(sspr[km+1]):
                            sspr[km] = (sspr[km-1]+sspr[km+1])/2.
                        else:
                            sspr[km] = sspr[km-1]  
                    except:
                        sspr[km] = sspr[km-1]  
            
            # 2/20/2020
            # we are removing the masked pixels and the nans and infs before fitting
            # lam larger => smoother baseline
            # ratio smaller => less neg. values
            asspr = nanarpls(sspr, lam=1000, ratio=0.005)
            bsspr = nanarpls(sspr, lam=1000, ratio=0.05)
            csspr = nanarpls(sspr, lam=100,  ratio=0.05)
            # repeat first baselinefit, but exclude the potential line region
            # this might work or not!!!
            sspr2 = sspr.copy()
            bxsel = np.where((svvr>bxrange[0])&(svvr<bxrange[1]))
            bxseln = np.where((svvr<bxrange[0])|(svvr>bxrange[1]))
            sspr2[bxsel] = np.nan
            dsspr = nanarpls(sspr2, lam=1000, ratio=0.005)
            # fill in the baseline values using interpolation 
            dfr = interpolate.interp1d(svvr[bxseln], dsspr[bxseln], fill_value='extrapolate', kind=bxmode)
            dsspr[bxsel] = dfr(svvr[bxsel])
            #print(dfr)
            #print(dsspr[bxsel])
            
            # test if we can expand the baseline to the full resolution.
            af = interpolate.interp1d(svvr, asspr, fill_value='extrapolate', kind='cubic')
            bf = interpolate.interp1d(svvr, bsspr, fill_value='extrapolate', kind='cubic')
            cf = interpolate.interp1d(svvr, csspr, fill_value='extrapolate', kind='cubic')
            # possible kinds: ‘linear’, ‘nearest’, ‘zero’, ‘slinear’, ‘quadratic’, ‘cubic’, ‘previous’, ‘next’
            # set near the top in bxmode
            df = interpolate.interp1d(svvr, dsspr, fill_value='extrapolate', kind=bxmode)
            nrpx = svvr.size
            # baseline for full spectrum
            assps = af(svv)
            bssps = bf(svv)
            cssps = cf(svv)
            dssps = df(svv)
            
            # replace the masked data in the signal array with the baseline fit data
            sig[smask>1] = bssps[smask>1]
            
            # prepare arrays for storage
            nres = 4
            svvrs = np.zeros(256)
            svvrs[0:nrpx] = svvr
            ssprs = np.zeros(256)
            ssprs[0:nrpx] = sspr
            smskr = np.zeros((256,16), dtype=np.int)
            smskr[0:nrpx,0] = smsk
            # create arrays of the reduced (resampled) baseline fits
            assprs = np.zeros(256)
            assprs[0:nrpx] = asspr
            bssprs = np.zeros(256)
            bssprs[0:nrpx] = bsspr
            cssprs = np.zeros(256)
            cssprs[0:nrpx] = csspr
            dssprs = np.zeros(256)
            dssprs[0:nrpx] = dsspr

            cmask[:,0:11] = np.vstack([smask, t1mask, t2mask, drumask, dhumask, ohumask, orumask, ordmask, ohdmask, drdmask, dhdmask]).swapaxes(0,1)
            amask[0:11] = np.vstack([sbad, t1bad, t2bad, drubad, dhubad, ohubad, orubad, ordbad, ohdbad, drdbad, dhdbad]).swapaxes(0,1)
            
            
            sig_wm = sig.copy()
            sigc = sig.copy()# = smrepMask(svv, sig, smask)
            
            # calculate some rms values
            if isinstance(rms1range,np.ndarray):
                rms1sel = np.where((svv.value>=rms1range[0])&(svv.value<=rms1range[1]))
                rms1vv = svv[rms1sel,]
                rms1sig = np.squeeze(sig[rms1sel,])
                rms1 = moment(rms1sig, moment=2, nan_policy='omit')
            else:
                rms1 = None
            
            # calculate some rms values
            if isinstance(rms2range,np.ndarray):
                rms2sel = np.where((svv.value>=rms2range[0])&(svv.value<=rms2range[1]))
                rms2vv = svv[rms2sel,]
                rms2sig = np.squeeze(sig[rms2sel,])
                rms2 = moment(rms2sig, moment=2, nan_policy='omit')
            else:
                rms2 = None
            
            if debug:
                print('Plotting spectrum: ', scan, sobsid)
                dsel = np.where((svv.value>anrange[0])&(svv.value<anrange[1]))[0]
                fig = plt.figure(figsize=(10,8))
                sp1 = plt.subplot(4,1,1)
                csig = np.squeeze(sig[rsel])
                csig[smaskr>0] = off
                pl1 = plt.plot(svv[rsel], csig-off, drawstyle='steps-post',label='sig',color='blue')
                plt.xlim(anrange)
                
                sp1 = plt.subplot(4,1,2)
                #pl1 = plt.plot(svv[rsel], smaskr, drawstyle='steps-post',label='smaskr',color='blue')
                plt.xlim(anrange)
                dsel = np.where((svv.value>anrange[0])&(svv.value<anrange[1]))[0]
                pl2 = plt.plot(svv[dsel], sig[dsel], drawstyle='steps-post',label='sig',color='blue')
                pl2 = plt.plot(svvr, sspr, drawstyle='steps-post',label='asspsr',color='blue')
                #pl2 = plt.plot(svvr, ssprb, drawstyle='steps-post',label='asspsr',color='purple')
                pl2 = plt.plot(svvr, asspr, drawstyle='steps-post',label='asspsr',color='orange')
                pl2 = plt.plot(svvr, bsspr, drawstyle='steps-post',label='bsspsr',color='red')
                pl2 = plt.plot(svvr, csspr, drawstyle='steps-post',label='csspsr',color='green')
                pl2 = plt.plot(svvr, dsspr, drawstyle='steps-post',label='dsspsr',color='olive', linewidth=3.0)
                
                
                sp1 = plt.subplot(4,1,3)                
                dsel = np.where((svv.value>anrange[0])&(svv.value<anrange[1]))[0]
                pl2 = plt.plot(svv[dsel], sig[dsel], drawstyle='steps-post',label='sig',color='blue')
                pl2 = plt.plot(svv[dsel], assps[dsel], drawstyle='steps-post',label='assps',color='orange')
                pl2 = plt.plot(svv[dsel], bssps[dsel], drawstyle='steps-post',label='bssps',color='red')
                pl2 = plt.plot(svv[dsel], cssps[dsel], drawstyle='steps-post',label='cssps',color='green')
                pl2 = plt.plot(svv[dsel], dssps[dsel], drawstyle='steps-post',label='dssps',color='olive', linewidth=3.0)
                plt.xlim(anrange)
                plt.legend()
                
                sp2 = plt.subplot(4,1,4)
                dsel = np.where((svv.value>anrange[0])&(svv.value<anrange[1]))[0]
                #pl2 = plt.plot(svv[dsel], sig[dsel], drawstyle='steps-post',label='sig',color='blue', alpha=0.3)
                pl2 = plt.plot(svv[dsel], sig[dsel]-assps[dsel], drawstyle='steps-post',label='sig-assps',color='orange')
                pl2 = plt.plot(svv[dsel], sig[dsel]-bssps[dsel], drawstyle='steps-post',label='sig-bssps',color='red')
                pl2 = plt.plot(svv[dsel], sig[dsel]-cssps[dsel], drawstyle='steps-post',label='sig-cssps',color='green')
                pl2 = plt.plot(svv[dsel], sig[dsel]-dssps[dsel], drawstyle='steps-post',label='sig-dssps',color='olive', linewidth=3.0)
                plt.xlim(anrange)
                plt.ylim([-10,30.])   # the fit/interpolation goes off the rails at the ends.
                plt.legend()
                
                #print(spec[100], sref[100], sig[100])
                #print(frac2, afile)
                #print(svvrs[0:nrpx])
                
            zu1s = np.zeros(50)
            zu1s[0:zu1.size] = zu1
            zu2s = np.zeros(50)
            zu2s[0:zu2.size] = zu2
            # save all the data from up in a record, which will be concatenated to form the master record
            sdat = np.array([(svv, sig, sigc, sref, ref1, ref2, frac, frac2, spec, dru, dhu, ohu, oru, ord, ohd, drd, dhd, smask, cmask, amask,
                              scan, sobsid, druobsid, dhuobsid, ohuobsid, oruobsid, ordobsid, ohdobsid, drdobsid, dhdobsid, 
                              gl, gb, stime, sTint, lin, repflag, repmode, 
                              svvrs, ssprs, smskr, assprs, bssprs, cssprs, dssprs, assps, bssps, cssps, dssps, nrpx, nres, vbrange, 
                              bxmode, sbcorr, calibtype, gainratio, calibunit, caliblevel, 
                              csp1, csp2, bcor1, bcor2, zu1s, zu2s,
                              Tsys, Tsys1, rep1, t1rec, Tsys2, rep2, t2rec, Thot, Tsky, rms1, rms1range, rms2, rms2range)], 
                              dtype=[
                              ('vv', 'f8', 1024),('spec', 'f8', 1024),('specc', 'f8', 1024),('sref', 'f8', 1024),('ref1', 'f8', 1024),('ref2', 'f8', 1024), 
                              ('frac', 'f8'),('frac2', 'f8'),('spraw', 'f8', 1024),  
                              ('dru', 'f8', 1024), ('dhu', 'f8', 1024), ('ohu', 'f8', 1024), ('oru', 'f8', 1024), ('ord', 'f8', 1024), ('ohd', 'f8', 1024), 
                              ('drd', 'f8', 1024), ('dhd', 'f8', 1024), ('mask','i4', 1024), ('cmask','i4', (1024,16)), ('amask','i4', 16),
                              ('scan', 'f8'),('sobsid', 'f8'),('druobsid', 'f8'),('dhuobsid', 'f8'),
                              ('ohuobsid', 'f8'),('oruobsid', 'f8'),('ordobsid', 'f8'),('ohdobsid', 'f8'),('drdobsid', 'f8'),('dhdobsid', 'f8'),
                              ('gl','f8'),('gb','f8'),('time','f8'),('Tint','f8'),('line','i4'),('repflag','i4', 3),('repmode','S16', 3),
                              ('vvr','f8', 256),('spr','f8',256),('mkr','f8',(256,16)),('aspr','f8',256),('bspr','f8',256),('cspr','f8',256),('dspr','f8',256),('asp','f8',1024),('bsp','f8',1024),('csp','f8',1024),('dsp','f8',1024),('nrpx','f8'),('nres','f8'),('vbrange','f8',2),
                              ('bxmode', 'S16'),('sbcorr','f8'), ('calibtype', 'S3'), ('gainratio','f8'), ('calibunit', 'S16'), ('caliblevel', 'S16'),
                              ('csp1','f8', 1024), ('csp2','f8', 1024), ('bcor1','f8', 1024), ('bcor2','f8', 1024), ('zu1','f8', 50), ('zu2','f8', 50), 
                              ('Tsys', 'f8', 1024),('Tsys1', 'f8', 1024),('Trep1', 'S1'),('Trec1', 'i4'),('Tsys2', 'f8', 1024),('Trep2', 'S1'),('Trec2', 'i4'),('Thot', 'f8'),('Tsky', 'f8'),
                              ('rms1','f8'), ('rms1range','f8',2), ('rms2','f8'), ('rms2range','f8',2)])
            # 
            # We should also save the data in the format of the original file with the raw spectrum
            # replaced by the calibrated spectrum.
            # we have the primary header with empyt data section
            #  and the first extension with the table section. In latter is where the spectrum will end up.
            hd0 = srs.getHeader0()
            hd1 = srs.getHeader()
            dd0 = srs.getRawData0()
            dd1 = srs.getRawData()
            cols = srs.getDataColumns()
            
            omask1 = smask + drumask + dhumask + ohumask + orumask + ordmask + ohdmask + drdmask + dhdmask
            omask1[np.where(omask1>1)] = 1
            tmask1 = t1mask + t2mask
            sg = np.zeros([3,1024])
            sg[lin,:]=np.squeeze(sig)
            mcol1 = fits.Column(name='SPEC',  format='1024E', array=sg)
            sr = np.zeros([3,1024])
            sr[lin,:]=np.squeeze(sref)
            mcol2 = fits.Column(name='RefSP', format='1024E', array=sr)
            ts = np.zeros([3,1024])
            ts[lin,:]=np.squeeze(Tsys)
            mcol3 = fits.Column(name='TSYSs',  format='1024E', array=ts)
            om = np.zeros([3,1024], dtype=np.int)
            om[lin,:]=np.squeeze(omask1)
            mcol4 = fits.Column(name='SMASK', format='1024J', array=om)
            tm = np.zeros([3,1024], dtype=np.int)
            tm[lin,:]=np.squeeze(tmask1)
            mcol5 = fits.Column(name='TMASK', format='1024J', array=tm)
            aas = np.zeros([3,1024])
            aas[lin,:]=np.squeeze(assps)
            mcol6a = fits.Column(name='BL_CORR1', format='1024E', array=aas)
            bs = np.zeros([3,1024])
            bs[lin,:]=np.squeeze(bssps)
            mcol6b = fits.Column(name='BL_CORR2', format='1024E', array=bs)
            cs = np.zeros([3,1024])
            cs[lin,:]=np.squeeze(cssps)
            mcol6c = fits.Column(name='BL_CORR3', format='1024E', array=cs)
            bc1 = np.zeros([3,1024])
            bc1[lin,:]=np.squeeze(bcor1)
            mcol7 = fits.Column(name='BCOR1', format='1024E', array=bc1)
            bc2 = np.zeros([3,1024])
            bc2[lin,:]=np.squeeze(bcor2)
            mcol8 = fits.Column(name='BCOR2', format='1024E', array=bc2)
            sgc = np.zeros([3,1024])
            sgc[lin,:]=np.squeeze(sigc)
            mcol9 = fits.Column(name='SPECC',  format='1024E', array=sgc)
            new_columns = cols + mcol1 + mcol2 + mcol3 + mcol4 + mcol5 + mcol6a + mcol6b + mcol6c + mcol7 + mcol8 + mcol9
            new_hdu = fits.BinTableHDU.from_columns(new_columns)

            # save data in new file (old: afile)
            head, tail = ntpath.split(afile)
            oroot = 'aapr%i/'%(lin)
            if not os.path.isdir(sto2_path + oroot): os.mkdir(sto2_path + oroot)
            if not os.path.isdir(sto2_path + oroot + '%s/'%(cdirnum)): os.mkdir(sto2_path + oroot + '%s/'%(cdirnum))
            ofile = sto2_path + oroot + '%s/'%(cdirnum) + tail
            fits.writeto(ofile, dd0, hd0, overwrite=True)
            hd1['history'] = 'added calibrated data with updated mask; %s; V. Tolls'%(datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S'))
            hd1['history'] = 'added baseline fit data'
            hd1['history'] = 'added reference spectrum'
            hd1['history'] = 'despiked pixel replaced with interpolated value.'
            hd1['level1']  = ('VERSION %s'%(dversion), 'Applied on %s'%(datetime.datetime.now().strftime('%d %b %Y')))
            fits.append(ofile, new_hdu.data, hd1)
    
            if init: 
                sdata = sdat
                init = False
            else: 
                sdata = np.vstack((sdata,sdat))
    
    
    
    
    
    
    
    # final action at end of loop
    cnt += 1
    ##+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
    

# create numpy file:
# can be reloaded with: 
# npzfile = np.load(outfile)    # load file
# print(npzfile.files)          # print variables
# print(npzfile['variable'])    # retrieve variable
if saveflag:
    source = 'EtaCar2_OTF_L0.7-L1.0_processing_v4.ipynb'
    nfile = ofileroot+add+'_%i_%i_v2_%s.npz'%(stdir, endir, s_today)   # numpy archive file name
    np.savez(nfile, xfile=xfile, sdata=sdata, cline=cline, cline2=cline2, badotfs=badotfs, version='v2', source=source,
             pos0=pos0, l0=l0, b0=b0, lin=lin, target=target, ctarget=ctarget)

    print('saved: ', nfile)

<ipython-input-1-8891ec6e3af1>
Last execution: Tue May 19 16:46:48 2020
Eta Car: 287.58968,-0.63202
Eta Car: 10h45m00.0s -59d41m00.0s
Processing data for line:  CII 2
reading excel data analysis sheet.
Index(['drudir', '    dru', 'dhudir', '   dhu', 'dhucor', 'ohudir', '   ohu',
       'orudir', '   oru', 'orucor', 'spdir', '  spec', ' spec2', 'orddir',
       '   ord', 'ordcor', 'ohddir', '   ohd', 'dhddir', '   dhd', 'drddir',
       '   drd', 'des ref1', 'des ref2', 'brange1[0]', 'brange1[1]',
       'brange2[0]', 'brange2[1]', 'order1', 'order2', 'despike', 'despike2'],
      dtype='object')
(96,)
mapdirs:  [3801 3803 3805 3807 3809 3811 3813 3815 3817 3819 3821 3823 3825 3827
 3829 3831 3833 3835 3837 3839 3841 3843 3845 3847 3849 3851 3853 3855
 3857 3859 3861 3863 3865 3867 3869 3871 3873 3875 3877 3879 3881 3883
 3885 3887 3889 3891 3893 3895 3897 3899 3901 3903 3909 3911 3913 3915
 3917 3919 3921 3923 3925 3927 3929 3931 3933 3935 3937 3939 3941 3943
 3945 3947 3949 3951 3953 